# Batch Lever Press Analysis
## Notebook for preprocessing and analyzing coactivity spine data

In [ ]:
import os
import numpy as np
from Lab_Analyses.Spine_Analysis.preprocess_spine_data import organize_dual_spine_data
from Lab_Analyses.Spine_Analysis.grouped_coactivity_analysis import grouped_coactivity_analysis
from Lab_Analyses.Spine_Analysis.coactivity_plasticity_analysis import Coactivity_Plasticity
from Lab_Analyses.Utilities.save_load_pickle import load_pickle

### Datasets
Specify mice which mice to be processed and analyzed

In [ ]:
# Mouse ids
id_1 = "JW082"
id_2 = "JW083"
id_3 = "JW090"
id_4 = "JW092"
id_5 = "JW126"
id_6 = "JW129"
id_7 = "JW130"

mouse_list = [id_1, id_2, id_3, id_4, id_5, id_6, id_7]
#mouse_list = [id_4, id_6]

### Preprocessing
Preprocess spine datasets

In [ ]:
# Do you want to reprocess the data
reprocess = False
channels = {"GluSnFr": "GreenCh", "Calcium": "RedCh"}
fov_type = "apical"

# Process data
if reprocess:
    for mouse in mouse_list:
        organize_dual_spine_data(mouse, channels, fov_type=fov_type, redetection=True, save=True, structural=True)

### Grouped Coactivity Analysis
Perform coactivity analysis across all mice and FOVs for a given set of imaging sessions

In [ ]:
# Set up parameters
## Save path
group_path = r'C:\Users\Jake\Desktop\Analyzed_data\grouped'
experiment_dir = "Dual_Spine_Imaging"
coactivity_dir = "Coactivity_Data"
coactivity_path = os.path.join(group_path, experiment_dir, coactivity_dir)

## Analysis parameters
reanalyze = True
save = True
days = ["Early", "Middle", "Late"]
#days = ["Middle"]
followup = True
zscore = False
volume_norm = True
fov_type = "apical"

coactivity_data = {}
for day in days:
    print(f"-{day}")
    coactivity = grouped_coactivity_analysis(
        mouse_list,
        day,
        followup=followup,
        fov_type=fov_type,
        activity_window=(-2,4),
        zscore=zscore,
        volume_norm=volume_norm,
        save=save,
        save_path=coactivity_path,
    )

    coactivity_data[day] = coactivity

group_path = r'C:\Users\Jake\Desktop\Analyzed_data\grouped'
experiment_dir = "Dual_Spine_Imaging"
coactivity_dir = "Coactivity_Data"
coactivity_path = os.path.join(group_path, experiment_dir, coactivity_dir)

load_coactivity = True
if load_coactivity:
    names = ["Early_dFoF_norm_spine_coactivity_data", "Middle_dFoF_norm_spine_coactivity_data", "Late_dFoF_norm_spine_coactivity_data"]
    data = load_pickle(names, coactivity_path)
    days = ["Early", "Middle", "Late"]
    coactivity_data = dict(zip(days, data))

### Coactivity Plasticity Analysis
Perform analysis to relate subsequent plasticity to coactivity 

In [ ]:
## Save path
group_path = r'C:\Users\Jake\Desktop\Analyzed_data\grouped'
experiment_dir = "Dual_Spine_Imaging"
plasticity_dir = "Coactivity_Plasticity"
plasticity_path = os.path.join(group_path, experiment_dir, plasticity_dir)

In [ ]:

## Analysis parameters
threshold = 0.5
exclude = "Shaft Spine"
save = True

plasticity_data = {}
for key, value in coactivity_data.items():
    plasticity = Coactivity_Plasticity(
        value, threshold, exclude, vol_norm=False, save=save, save_path=plasticity_path
    )
    plasticity_data[key] = plasticity

### Visualize Coactivity Plasticity Data
Plot selected data and variables from individual datasets.
Use data generated above of load in a dataset

In [ ]:
# Select data to plot
load_data = False
initial_path = r"C:\Users\Jake\Desktop\Figures\grouped_data"
figure_path = os.path.join(initial_path, experiment_dir, plasticity_dir, "Early")
if not os.path.isdir(figure_path):
    os.makedirs(figure_path)
if load_data:
    load_name = "Early_dFoF_norm_0.5_coactivity_plasticity_data"
    plot_data = load_pickle([load_name], plasticity_path)[0]
else:
    plot_data = plasticity_data["Early"]


In [ ]:
# Print out variable names
for key in plot_data.__dict__.keys():
    print(key)

In [ ]:
initial_path = r"C:\Users\Jake\Desktop\Analyzed_data\individual"
behavior_path = os.path.join(initial_path, "JW082", "behavior")
final_day = sorted([x[0] for x in os.walk(behavior_path)])[-1]
final_day

In [ ]:
b = plot_data

In [ ]:
# Correlation Plots
plot_data.plot_volume_correlation(
    variable_name="conj_relative_onset",
    volume_type="relative_volume",
    CI=95,
    ytitle="Spine Calcium dF/F",
    xlim=None,
    ylim=None,
    face_color="mediumblue",
    edge_color="white",
    edge_width=1,
    s_alpha=0.4,
    line_color="mediumblue",
    line_width=1,
    log_trans=True,
    save=False,
    save_path=figure_path
)

In [ ]:
# Group mean activity plots
plot_data.plot_group_scatter_plot(
    variable_name="conj_dend_coactive_amplitude",
    group_type="plastic_spines",
    mean_type="mean",
    err_type="sem",
    figsize=(4,5),
    ytitle="Relative Coactivity Rate",
    ylim=None,
    s_colors=["darkorange", "forestgreen", "silver"],
    s_size=5,
    s_alpha=0.8,
    b_colors=["darkorange", "forestgreen", "silver"],
    b_edgecolors = "black",
    b_err_colors="black",
    b_width=0.5,
    b_linewidth=0,
    b_alpha=0.3,
    save=False,
    save_path=figure_path,
)

In [ ]:
print(np.sum(plot_data.enlarged_spines)/len(plot_data.enlarged_spines))
print(np.sum(plot_data.shrunken_spines)/len(plot_data.enlarged_spines))
print(np.sum(plot_data.stable_spines)/len(plot_data.enlarged_spines))
e= np.sum(plot_data.enlarged_spines*plot_data.movement_spines)/np.sum(plot_data.movement_spines)
s= np.sum(plot_data.shrunken_spines*plot_data.movement_spines)/np.sum(plot_data.movement_spines)
stab = np.sum(plot_data.stable_spines*plot_data.movement_spines)/np.sum(plot_data.movement_spines)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
sns.set_style("ticks")


plt.figure(figsize=(4,5))
plt.bar(x=[0,1,2], height=[e, s, stab], color=["darkorange", "forestgreen", "silver"])
plt.ylabel("Fraction of MRSs")
plt.xticks(ticks=[0,1,2], labels=["enlarged", "shrunken", "stable"])
plt.ylim(0, 0.5)
plt.tight_layout()
fname = os.path.join(figure_path, "frac_MRSs")
plt.savefig(fname + ".pdf")

In [ ]:

plot_data.plot_multi_group_scatter_plots(
    variable_name="conj_dend_coactive_amplitude",
    group_type="movement_spines",
    subgroup_type="plastic_spines",
    mean_type="mean",
    err_type="sem",
    figsize=(5,5),
    ytitle="Fraction of Dendritic Events",
    ylim=None,
    s_colors=["darkorange", "forestgreen", "silver"],
    s_alpha=0.8,
    s_size=5,
    b_colors=["darkorange", "forestgreen", "silver"],
    b_edgecolors="black",
    b_err_colors="black",
    b_width=0.2,
    b_linewidth=0,
    b_alpha=0.3,
    save=False,
    save_path=figure_path,
)

In [ ]:
plot_data.plot_group_spine_mean_traces(
    group_type="plastic_spines",
    trace_type="conj_dend_coactive_traces",
    exclude=[],
    avlines=[0],
    figsize=(5,5),
    colors=["darkorange", "forestgreen", "silver"], 
    ylim=None,
    save=False,
    save_path=figure_path
)

In [ ]:
plot_data.plot_spine_coactivity_distance(
    variable_name="relative_distance_movement_corr",
    group_type="plastic_spines",
    figsize=(5,5),
    colors=["darkorange", "forestgreen", "silver"],
    ylim=None,
    ytitle="Local Normalized Coactivity Rate",
    save=False,
    save_path=figure_path,
)

In [ ]:
plot_data.plot_histogram(
    variable="conj_relative_onset",
    bins=50,
    max_lim=None,
    avlines=(0,0),
    figsize=(5,5),
    color="blue",
    alpha=0.4,
    save=False,
    save_path=figure_path
)

In [ ]:
plot_data.plot_group_mean_heatmaps(
    trace_type="rwd_move_spine_traces",
    group_type="plastic_spines",
    figsize=(10,5),
    hmap_range=(0.2,1),
    center=None,
    sorted="peak",
    normalize=True,
    cmap="plasma",
    save=False,
    save_path=figure_path
)

In [ ]:
from Lab_Analyses.Spine_Analysis.spine_plotting import plot_sns_scatter_correlation




In [ ]:
original = np.log10(a.relative_volumes)
original = a.relative_volumes
norm = b.relative_volumes

In [ ]:
plot_sns_scatter_correlation(
    original,
    norm,
    CI=95,
    title="Relative Volume Comparison",
    xtitle="Original Relative Volume",
    ytitle="Normalized Relative Volume",
    marker_size=4,
    
)

In [ ]:
a = np.float64(5)

In [ ]:
if np.issubdtype(type(a), float):
    print("integer")

In [ ]:
a = 2
b = 6

diff = (b - a) / np.absolute(a)
diff

In [ ]:
on = np.nonzero(b == 1)[0]
off = np.nonzero(b == -1)[0]

In [ ]:
print(on)
print(off)

In [ ]:
if on[0] > off[0]:
    off = off[1:]
if len(on) > len(off):
    on = on[:-1]

In [ ]:
c = np.array([[1,2,0,4],
              [1,2,0,4]])

In [ ]:
d=np.sum(c, axis=0)

In [ ]:
np.nonzero(d)[0]